In [1]:
from utils import *
from k_means import *
from image_segmentation import *
from metrics import *


%matplotlib inline

# visualize_image('22090', 'train')


X, y = load_images('test', limit=True, max_limit=2)
print(y.shape)
print(X.shape)
# plt.imshow(y[0][0][:, :, 0])
    

# images = kmeans_segment_images(X, cache=True, load_from_cache=True)
# plt.imshow(images[0][1])
# print('>')
# print(images[0].shape)
# plt.imshow(y[0][0][:, :, 1], cmap='Greys_r') # boundries
# plt.imshow(y[0][0][:, :, 0])  # segmentation
# plt.figure()
# plt.axis('off')
# plt.imshow(y[0][0][:, :, 1]) # boundries

# f = f1_score_single(images[0], y[0][0][:, :, 0])
# print('f = {}'.format(f))

# ce = conditional_entropy(images[0], y[0][0][:, :, 0])
# print('conditional_entropy = {}'.format(ce))

# plt.figure()
# plt.imshow(y[0][4][:, :, 0])
# plt.figure()
# plt.imshow(y[0][0][:, :, 0])
# for k in range(images.shape[1]):
#     plt.figure()
#     plt.imshow(images[0, k, :, :])
#     print(" ==================== K = {} =================".format(k))
#     avgs = evaluate_k_means_entropy(images[:, k, :, :], y)
#     print("average = {} max = {} min = {}".format(np.average(avgs), np.max(avgs), np.min(avgs)))
    
#     print(" ==================== K = {} =================".format(k))
#     f = f1_score_all(images[:, k, :, :], y)
#     print(f)



# img = np.array(get_image('2092', 'train'))
# clusters, _ = segment(img)

# seg_image = clusters.reshape((img.shape[0], img.shape[1]))
# plt.imshow(seg_image)
# plt.axis('off')
# plt.savefig(f'./images/test.jpg')

# seg_image = np.rot90(seg_image, 2)
# seg_image = np.fliplr(seg_image)
# xx = np.arange(img.shape[0])
# yy = np.arange(img.shape[1])
# X, Y = np.meshgrid(yy, xx)

# fig = plt.figure()
# h = plt.contourf(X, Y, seg_image)
# plt.axis('off')
# plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, hspace = 0, wspace = 0)
# plt.savefig(f'./Images/{i}_{k}.jpg')


(2, 5, 481, 321, 2)
(2, 481, 321, 3)


In [ ]:
clustered_images = []
for i in range(X.shape[0] - 1):
    res = spectral_segment(X[i], n_clusters=11)
    print(res)
    plt.figure()
    plt.imshow(X[i])
    plt.figure()
    plt.imshow(res)
    clustered_images.append(res)
#     with open(f'./images/spectral_labels_{i}.pkl', 'wb+') as f:
#         pickle.dump(clustered_images[-1], f)
#     print(f'clustered image {i}')

/anaconda3/envs/cs231n/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/anaconda3/envs/cs231n/lib/python3.6/site-packages/sklearn/manifold/spectral_embedding_.py:234: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
